<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [4]:
import pandas as pd
import psycopg2
import warnings
warnings.filterwarnings('ignore', message = '.*SQLAlchemy connectable')

In [5]:
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432 

In [6]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [7]:
# текст запроса
query_3_1 = f'''select count(*)
                from public.vacancies
            '''


In [8]:
# результат запроса
vacancies_count = pd.read_sql_query(query_3_1, connection)
display(vacancies_count)

,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [9]:
# текст запроса
query_3_2 = f'''select count(*)
                from public.employers
            '''


In [10]:
# результат запроса
employers_count = pd.read_sql_query(query_3_2, connection)
display(employers_count)

,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [11]:
# текст запроса
query_3_3 = f'''select count(*)
                from public.areas
            '''

In [12]:
# результат запроса
areas_count = pd.read_sql_query(query_3_3, connection)
display(areas_count)

,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [13]:
# текст запроса
query_3_4 = f'''select count(*)
                from public.industries
            '''

In [14]:
# результат запроса
industries_count = pd.read_sql_query(query_3_4, connection)
display(industries_count)

,count
0,294


***

### Выводы по предварительному анализу данных
Итого мы имеем:
* данные о примерно 50 000 вакансий
* порядка 25 000 работодателей (то есть, в среднем по две вакансии на одно предприятие)
* порядка 1 500 регионов (значит, это скорее города, чем регионы в административном смысле)
* около 300 сфер деятельности (похоже, что классификатор сфер составлен очень детально)

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [15]:
# текст запроса
query_4_1 = f'''select a.name as area, count(*) as cnt
                from public.areas a 
                left join public.vacancies v on a.id = v.area_id
                group by a.id
                order by cnt desc
            '''

In [16]:
# результат запроса
vacancies_by_area = pd.read_sql_query(query_4_1, connection)
display(vacancies_by_area)

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
1357,Слуцк,1
1358,Ува,1
1359,Тихорецк,1
1360,Олекминск,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [17]:
# текст запроса
query_4_2 = f'''select count(*)
                from public.vacancies v
                where (v.salary_from is not null) or (v.salary_to is not null)
            '''

In [18]:
# результат запроса
vacancies_with_salary = pd.read_sql_query(query_4_2, connection)
display(vacancies_with_salary)

,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [19]:
# текст запроса
query_4_3 = f'''select round(avg(v.salary_from)), round(avg(v.salary_to))
                from public.vacancies v
            '''

In [20]:
# результат запроса
vacancies_avg_salary = pd.read_sql_query(query_4_3, connection)
display(vacancies_avg_salary)

,round,round
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [21]:
# текст запроса
query_4_4 = f'''select v.schedule, v.employment, count(*) as cnt
                from public.vacancies v
                group by v.schedule, v.employment
                order by cnt desc
            '''

In [22]:
# результат запроса
vacancies_by_schedule_employment = pd.read_sql_query(query_4_4, connection)
display(vacancies_by_schedule_employment)

,schedule,employment,cnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [23]:
# текст запроса
query_4_5 = f'''select v.experience, count(*) as cnt
                from public.vacancies v
                group by v.experience
                order by cnt
            '''

In [24]:
# результат запроса
vacancies_by_experience = pd.read_sql_query(query_4_5, connection)
display(vacancies_by_experience)

,experience,cnt
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

### Выводы по детальному анализу вакансий
* Наибольшее число вакансий ожидаемо приходится на крупнейшие города (Москва, Санкт-Петербург, Новосибирск). Наличие в списке Минска и Алматы подсказывает нам, что сервис работает не только в России, но также (как минимум) в Беларуси и Казахстане.
* Примерно в половине вакансий указана хотя бы одна граница зарплатного диапазона.
* Средние границы зарплатного диапазона простираются примерно от 70 до 110 тысяч (я бы уточнил, что рублей, т.к. если там подмешаны суммы в других валютах, то результаты будут искажены)
* Ожидаемо значительное большинство (около 70%) вакансий предполагают полный день и полную занятость. Ещё около 15% предполагают удаленную работу при полной занятости.
* Около половины вакансий требуют опыт от 1 года до 3 лет, ещё примерно треть - от 3 до 6 лет. Примерно 15% вакансий подходят для кандидатов без опыта работы.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [25]:
# текст запроса
query_5_1 = f'''select e.name as employer, count(*) as cnt
                from public.employers e 
                left join public.vacancies v on e.id = v.employer_id
                group by e.id
                order by cnt desc
            '''

In [26]:
# результат запроса
vacancies_by_employer = pd.read_sql_query(query_5_1, connection)
display(vacancies_by_employer)

,employer,cnt
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331
...,...,...
23496,Тепло Людям,1
23497,Шанхай Спецтехника,1
23498,Уварова Елена,1
23499,Бугров Денис Владимирович,1


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [27]:
# текст запроса
query_5_2 = f'''select a.name as area, count(distinct e.id) as cnt_e, count(distinct v.id) as cnt_v
                from public.areas a 
                left join public.employers e on a.id = e.area
                left join public.vacancies v on a.id = v.area_id
                group by a.id
                order by cnt_v, cnt_e desc
            '''

In [28]:
# результат запроса
vacancies_by_area_and_employer = pd.read_sql_query(query_5_2, connection)
display(vacancies_by_area_and_employer)

,area,cnt_e,cnt_v
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Ростовская область,18,0
...,...,...,...
1357,Алматы,721,1892
1358,Новосибирск,573,2006
1359,Минск,1115,2112
1360,Санкт-Петербург,2217,2851


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [29]:
# текст запроса
query_5_3 = f'''select e.name, count(distinct a.id) as cnt
                from public.employers e
                left join public.vacancies v on v.employer_id = e.id
                left join public.areas a on a.id = v.area_id
                group by e.id
                order by cnt desc
            '''

In [30]:
# результат запроса
areas_by_employer = pd.read_sql_query(query_5_3, connection)
display(areas_by_employer)

,name,cnt
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
23496,Но Смокинг,0
23497,МАСТЕРСКАЯ CREED,0
23498,Dzing Finance,0
23499,КОРС,0


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [31]:
# текст запроса
query_5_4 = f'''select count(distinct e.id) from public.employers e
            left join public.employers_industries ei on ei.employer_id = e.id
            where ei.employer_id is null
            '''

In [32]:
# результат запроса
employers_no_industry_count = pd.read_sql_query(query_5_4, connection)
display(employers_no_industry_count)

,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [33]:
# текст запроса
query_5_5 = f'''select e.name, count(distinct ei.industry_id) from public.employers e
            join public.employers_industries ei on ei.employer_id = e.id
            group by e.id
            having count(distinct ei.industry_id) = 4
            order by e.name
            '''

In [34]:
# результат запроса
employers_4_industry = pd.read_sql_query(query_5_5, connection)
display(employers_4_industry)

,name,count
0,101 Интернет,4
1,21vek.by,4
2,2ГИС,4
3,2К,4
4,4 пикселя +,4
...,...,...
1133,ЮРИОН,4
1134,ЮТИП Технологии,4
1135,ЯКласс,4
1136,ЯрНео,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [35]:
# текст запроса
query_5_6 = f'''select count(distinct e.id) from public.employers e
            join public.employers_industries ei on ei.employer_id = e.id
            join public.industries i on i.id = ei.industry_id
            where i.name = 'Разработка программного обеспечения'
            '''

In [36]:
# результат запроса
employers_razrabotka = pd.read_sql_query(query_5_6, connection)
display(employers_razrabotka)

,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [37]:
# код для получения списка городов-милионников

import requests
from bs4 import BeautifulSoup
url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России'
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')

td_list = page.find_all('table')[1].find_all('td')
city_tuple = tuple([td.text for td in td_list][1::6])
print(city_tuple)


('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Красноярск', 'Челябинск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Краснодар', 'Омск', 'Воронеж', 'Пермь', 'Волгоград')


In [38]:
# текст запроса
query_5_7 = f'''select a.name, count(distinct v.id) as cnt
                from public.employers e
                left join public.vacancies v on v.employer_id = e.id
                left join public.areas a on a.id = v.area_id
                where e.name = 'Яндекс' and a.name in {city_tuple}
                group by a.id
                union all
                select 'Total', count(distinct v.id)
                from public.employers e
                left join public.vacancies v on v.employer_id = e.id
                left join public.areas a on a.id = v.area_id
                where e.name = 'Яндекс' and a.name in {city_tuple}
                order by cnt
            '''

In [39]:
# результат запроса
yandex_vacancies = pd.read_sql_query(query_5_7, connection)
display(yandex_vacancies)

,name,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Самара,26
8,Уфа,26
9,Краснодар,30


***

### Выводы по анализу работодателей
* Яндекс является крупнейшим работодателем с большим отрывом от второго места (почти 2000 вакансий против 500 у Ростелекома)
* Вакансии Яндекса также максимально широко представлены (в 181 "регионе")
* Видно, что справочник регионов (area) составлен не очень удобно, в него включены вперемешку как страны (например, Россия), так регионы (например, Московская область) и конкретные города, это может затруднить в будущем анализ по этому показателю
* Примерно у трети работодателей совсем не указана сфера деятельности
* Из тех примерно 15 000 компаний, для которых сфера деятельности указана, более чем каждый пятый указывает в качестве одной из таких сфер разработку ПО - что может говорить о высокой популярности hh.ru среди IT-компаний

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [40]:
# текст запроса
query_6_1 = f'''select count(*) from public.vacancies v
            where lower(v.name) like '%data%' or lower(v.name) like '%данн%'
            '''

In [41]:
# результат запроса
data_vacancies_count = pd.read_sql_query(query_6_1, connection)
display(data_vacancies_count)

,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [42]:
# текст запроса
query_6_2 = f'''select count(*) from public.vacancies v
            where (lower(v.name) like '%data scientist%' or
            lower(v.name) like '%data science%' or
            lower(v.name) like '%исследователь данных%' or
            (v.name like '%ML%' 
            and v.name not like '%HTML%') or
            lower(v.name) like '%machine learning%' or
            lower(v.name) like '%машинн%обучен%')
            and 
            (lower(v.name) like '%junior%'
            or v.experience = 'Нет опыта'
            or v.schedule = 'Стажировка')
            '''

In [43]:
# результат запроса
ds_vacancies_count = pd.read_sql_query(query_6_2, connection)
display(ds_vacancies_count)

,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [44]:
# текст запроса
query_6_3 = f'''select count(*) from public.vacancies v
            where (lower(v.name) like '%data scientist%' or
            lower(v.name) like '%data science%' or
            lower(v.name) like '%исследователь данных%' or
            (v.name like '%ML%' 
            and v.name not like '%HTML%') or
            lower(v.name) like '%machine learning%' or
            lower(v.name) like '%машинн%обучен%')
            and 
            (lower(v.key_skills) like '%sql%'
            or lower(v.key_skills) like '%postgres%')
            '''

In [45]:
# результат запроса
ds_skills_vacancies_count = pd.read_sql_query(query_6_3, connection)
display(ds_skills_vacancies_count)

,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [46]:
# текст запроса
query_6_4 = f'''select count(*) from public.vacancies v
            where (lower(v.name) like '%data scientist%' or
            lower(v.name) like '%data science%' or
            lower(v.name) like '%исследователь данных%' or
            (v.name like '%ML%' 
            and v.name not like '%HTML%') or
            lower(v.name) like '%machine learning%' or
            lower(v.name) like '%машинн%обучен%')
            and 
            (lower(v.key_skills) like '%python%')
            '''

In [47]:
# результат запроса
ds_python_vacancies_count = pd.read_sql_query(query_6_4, connection)
display(ds_python_vacancies_count)

,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [48]:
# текст запроса
query_6_5 = f'''select
            round(avg((length(v.key_skills) - length(replace(v.key_skills,'\t',''))) + 1), 2)
            from public.vacancies v
            where ((lower(v.name) like '%data scientist%' or
            lower(v.name) like '%data science%' or
            lower(v.name) like '%исследователь данных%' or
            (v.name like '%ML%' 
            and v.name not like '%HTML%') or
            lower(v.name) like '%machine learning%' or
            lower(v.name) like '%машинн%обучен%'))
            and v.key_skills is not null
            '''

In [49]:
# результат запроса

ds_skill_count = pd.read_sql_query(query_6_5, connection)
display(ds_skill_count)

,round
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [50]:
# текст запроса
query_6_6 = f'''select
            v.experience, round(avg((coalesce(salary_from, salary_to) + coalesce(salary_to, salary_from))/2))
            from public.vacancies v
            where ((lower(v.name) like '%data scientist%' or
            lower(v.name) like '%data science%' or
            lower(v.name) like '%исследователь данных%' or
            (v.name like '%ML%' 
            and v.name not like '%HTML%') or
            lower(v.name) like '%machine learning%' or
            lower(v.name) like '%машинн%обучен%'))
            and (salary_from is not null
            or salary_to is not null)
            group by v.experience
            order by 2
            '''

In [51]:
# результат запроса
ds_avg_salary = pd.read_sql_query(query_6_6, connection)
display(ds_avg_salary)

,experience,round
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

### Выводы по предметному анализу
* В представленной выборке около 500 вакансий для DS. Только 10% из них рассчитаны на специалистов без опыта.
* Требование знания Python для DS очень широко распространено (около 70% вакансий)
* Требование знания SQL или PostgreSQL чуть менее распространено (примерно 40% выборки)
* Заработная плата специалистов DS быстро растёт с увеличением опыта: при опыте от 1 года - вдвое, при опыте от 3 лет - более чем втрое по сравнению с зарплатой начинающих дата-сайентистов
* В выборке довольно подробно описаны навыки, необходимые специалистам: свыше 6 навыков в среднем на одну вакансию


# Общий вывод по проекту
* Нам предоставлена выборка из около 50000 вакансий
* Крупнейшим работодателем (и по числу вакансий, и в разрезе регионов) является Яндекс (по всей видимости, к вакансиям Яндекса относятся и вакансии в его дочерних проектах, таких как такси, еда, доставка и др.)
* Почти 2000 вакансий так или иначе относятся к работе с данными, около 500 относятся к Data Science
* Начальная зарплата для специалистов DS находится на уровне нижней границы среднего диапазона зарплат по выборке в целом (чуть более 70000 рублей), однако зарплата опытных специалистов DS более чем вдвое превосходит среднюю зарплату по выборке в целом (свыше 240000 рублей)
* Справочник регионов устроен неоптимально, в нём смешаны страны, регионы и города. Для дальнейшего анализа было бы полезно преобразовать его в иерархический или разделить на три отдельных справочника.

### Дальнейшие направления исследования
* Было бы интересно изучить, какие навыки вообще требуются для специалистов DS (мы уже знаем, что это поле заполняют тщательно - свыше 6 навыков на вакансию в среднем)
* Можно посмотреть, как различаются требуемые навыки в зависимости от уровня опыта

In [57]:
# прочитаем данные об опыте и навыках, для удобства восприятия заменим табуляцию на ;
# сразу отсеем строки, где опыт или навыки не указаны

query_add = f'''select
            v.experience, replace(v.key_skills, '\t', ';') as all_skills
            from public.vacancies v
            where ((lower(v.name) like '%data scientist%' or
            lower(v.name) like '%data science%' or
            lower(v.name) like '%исследователь данных%' or
            (v.name like '%ML%' 
            and v.name not like '%HTML%') or
            lower(v.name) like '%machine learning%' or
            lower(v.name) like '%машинн%обучен%'))
            and (v.experience is not null) 
            and (v.key_skills is not null)
            '''

In [68]:
# результат запроса
ds_experience_skills = pd.read_sql_query(query_add, connection)
display(ds_experience_skills)

,experience,all_skills
0,От 1 года до 3 лет,Python;SQL;Работа в команде;OpenCV;ML
1,От 1 года до 3 лет,Oracle Pl/SQL;Базы данных;Machine Learning;Маш...
2,От 1 года до 3 лет,Python;OpenCV;Tensorflow;Computer Vision;Нейро...
3,От 3 до 6 лет,Python;Flask;Linux;AWS;Git
4,От 1 года до 3 лет,Python;SQL;NLP;Scikit-learn;Machine Learning
...,...,...
426,От 3 до 6 лет,Python;Numpy;PyTorch;Pandas;Математический ана...
427,От 3 до 6 лет,Python;Scikit-learn;Pandas;Git;Numpy
428,От 3 до 6 лет,Qt;ООП;C++;MS Visual Studio;Разработка компьют...
429,От 3 до 6 лет,SCALA;SQL;Математический анализ;Статистический...


In [69]:
# сначала сформируем список всех навыков без повторов
import collections as col

full_skill_list = []    # полный список навыков 
top_skill_list = []     # список самых популярных навыков
top_skill_agg = {}      # словарь, с помощью которого мы зададим агрегационную функцию
n = 15                  # сколько навыков включаем в топ

sql_skill_list = (ds_experience_skills['all_skills']).to_list()     # читаем результаты из sql в список

for skill_list in sql_skill_list:                                   # формируем список навыков с повторами
    for skill in skill_list.split(sep=';'):
        full_skill_list.append(skill.lower())
        
skill_counter = col.Counter(full_skill_list).most_common(n)         # считаем навыки и берём top n

for item in skill_counter:                                          # формируем список top n навыков
    top_skill_list.append(item[0])

# теперь выполним парсинг навыков в отдельные колонки-мигалки
# и заодно зададим агрегационную функцию mean для всех этих колонок
for skill in top_skill_list:
    ds_experience_skills[skill] = ds_experience_skills['all_skills'].apply(lambda x: False if x.lower().find(skill) == -1 else True)
    top_skill_agg.setdefault(skill, 'mean')

# удаляем изначальную колонку all_skills и агрегируем данные по опыту работы
ds_experience_skills.drop('all_skills', axis=1, inplace=True)
ds_experience_skills_group = ds_experience_skills.groupby('experience').agg(top_skill_agg)

display(ds_experience_skills_group)

,python,sql,machine learning,git,data science,математическая статистика,pytorch,pandas,data analysis,ml,английский язык,linux,анализ данных,tensorflow,nlp
experience,,,,,,,,,,,,,,,
Более 6 лет,0.606061,0.060606,0.606061,0.212121,0.060606,0.000000,0.060606,0.484848,0.515152,0.181818,0.000000,0.000000,0.000000,0.060606,0.030303
Нет опыта,0.884615,0.384615,0.500000,0.115385,0.230769,0.192308,0.153846,0.038462,0.076923,0.307692,0.115385,0.076923,0.115385,0.153846,0.153846
От 1 года до 3 лет,0.809278,0.670103,0.257732,0.159794,0.113402,0.216495,0.139175,0.087629,0.118557,0.113402,0.113402,0.118557,0.113402,0.103093,0.113402
От 3 до 6 лет,0.848315,0.331461,0.325843,0.162921,0.191011,0.084270,0.157303,0.129213,0.067416,0.213483,0.129213,0.123596,0.117978,0.084270,0.101124


In [73]:
# строим тепловую карту востребованности навыков в зависимости от опыта работы
import plotly.express as px

fig = px.imshow(ds_experience_skills_group, title='Распределение навыков по опыту работы')
fig.update_yaxes(categoryorder='array', categoryarray= ['Нет опыта','От 1 года до 3 лет','От 3 до 6 лет','Более 6 лет'])
fig.update_xaxes(title_text='skill')
fig.show()
fig.write_html("graphs/heat_experience_skills.html")



Видим, что знание Python является максимально востребованным навыком для кандидатов на DS-вакансии. Любопытно, что для специалистов с большим стажем "популярность" Python снижается, хотя остаётся довольно высокой. Знание SQL больше всего востребовано для кандидатов со стажем от 1 до 3 лет. А вот, например, знание pandas и навыки data analysis максимально востребованы у опытных специалистов со стажем более 6 лет.

Впрочем, очевидно, что с этими данными предстоит ещё поработать. Например, тут есть отдельно machine learning и есть ml, что, очевидно, одно и то же. За пределами top 15 тоже наверняка найдётся много синонимов, которые стоит объединить.

In [ ]:
# закрываем соединение
connection.close()